# **Bibliotecas**

---

Nesta seção do código são importadas todas a bibliotecas necessárias que serão utilizadas pelo resto do programa. Em python para importar uma biblioteca tipicamente é atribuido um apelido para ela que será utilizada toda vez que quisermos utilizar uma função dela. Por exempo o apelido de numpy é np, ou seja, toda vez que aparecer np no código estamos utilizando uma função da biblioteca numpy. 

In [ ]:
import numpy as np

import matplotlib.pyplot as plt

#import qmcpy as qp

import time

import collections.abc

from multiprocessing import Pool

from multiprocessing import Process

from bibliotecas.argon_param import *

import bibliotecas.geofunc2 as g2

import bibliotecas.differential_equations as diff

import pandas as pd

# **Pontos de Emissão**

---

Nesta seção são declaradas inicialmente as variaveis que definem as coordenadas tanto do ponto de emissão(x0,y0,z0), quanto as coordenadas do centro do sensor ARAPUCA (dev_x1, dev_y1, dev_z1). Nesta seção as variaveis recebem um valor inicial, porém esses valores podem mudar em outras seções.

In [ ]:
# point of emission

x0 = 80

y0 = Ly/2+ed

z0 = Lz/2+ed

# detector device center position

dev_x1 = ed

dev_y1 = Ly/2+ed

dev_z1 = Lz/2+ed

# **Imagens da difusão**

---

Essa sessão quando rodada irá gerar as imagens da difusão dos fótons no argonio liquido. Todas as imagens geradas vão para a pasta Imagens.

In [ ]:
from pylab import meshgrid,cm,imshow,contour,clabel,colorbar,axis,title,show

x0 = Lx/4.
y0 = 20.
z0 = Lz/2.

x = np.arange(0.0,Lx,5)
y = np.arange(0.0,Ly,5)
X,Y = meshgrid(x, y)

for i in range(10):
    inst = 0.1+2*i
    
    
 # grid of point
    Z = diff.draw2d(inst, x, y, z0, x0, y0, z0) # evaluation of the function on the grid
    if i==0:
        z_max = np.max(Z)
    im = imshow(Z, extent=[0,x.max(),0,y.max()], cmap=plt.colormaps['viridis']) # drawing the function
    # adding the Contour lines with labels
    #cset = contour(Z,arange(-1,1.5,0.2),linewidths=2,cmap=cm.Set2)
    #clabel(cset,inline=True,fmt='%1.1f',fontsize=10)
    
    colorbar(im) # adding the colobar on the right
    # latex fashion title
    #title('$z=(1-x^2+y^3) e^{-(x^2+y^2)/2}$')
    plt.savefig('Imagens/fig'+str(i)+'.png') #Salva na pasta imagens
    plt.clf()
    print(i)



# **Histograma** 

---

Esse código irá gerar o histograma da quantidade de fotons coletados pelo ARAPUCA ao longo do tempo que é dado em nanossegundos. Aqui para obter diferentes resultados será necessário mudar as coordenadas (x0,y0,z0) do ponto de emissão, de acordo com os arquivos .csv na pasta "Tabelas de emissao". No total serão 5 pontos de emissão diferentes (A,B,C,D,E). Para cada um desses pontos será necessário manter y0 e z0 constantes, e aumentar a distancia x0 em 100cm indo de 1cm até 700cm, totalizando 8 histogramas para cada um dos 5 pontos de emissão. (ver a imagem ilustração_pontos_emissão.png)

Obs:. Para salvar os histogramas, salve na pasta correspondente ao ponto de emissão da seguinte forma: ponto(distancia_x). Ex: A(100cm), A(200cm) etc.

In [ ]:
#coordenadas do ponto de emissão
x0 = v*4
y0 = Ly/2
z0 = Lz/2

start_time = time.time()
t_max = 25
calc_flux = diff.flux_detector(t_max, 0, y0, z0, x0, y0, z0, 1, 0, 0) #flux_detector(t_max, dev_x1, dev_y1, dev_z1, x0, y0, z0, 1, 0, 0)
t = np.linspace(0,t_max, len(calc_flux))
plt.plot(t, calc_flux)
plt.yscale('log')
print('running time = '+ f'{time.time()-start_time}')

# **Gráfico Exponencial**

---
Este código quando rodado irá gerar o gráfico do numero total de fótons coletados pela ARAPUCA pela distância do ponto de emissão.

In [ ]:
#Extrai os pontos de emissão da tabela 
data_points = pd.read_csv('Tabelas de emissao/A.csv', sep=";") 
emission = []

#Passa as coordenadas x,y,z dos pontos de emissão para a matriz emission
for index, row in data_points.iterrows():
    xe = row['x']
    ye = row['y']
    ze = row['z']
    emission.append([xe,ye,ze])
    
total_photons = np.zeros(len(emission))
t_max = 50

#Calcula o numero total de fotons para o ponto de emissão especificado
def run_points(xxx,yyy,zzz):
    global x0
    x0 = xxx
    global y0
    y0 = yyy
    global z0
    z0 = zzz
    
    return diff.total_flow_detector(t_max, dev_x1, dev_y1, dev_z1, x0, y0, z0, 1, 0, 0)

#Armazena o numero total de fotons na lista total_photons usando multiprocessamento
def pool_handler():
    p = Pool()
    global total_photons
    total_photons = p.starmap(run_points, emission)
    p.close()
    p.join()
    

start_time = time.time()

if __name__ == '__main__':
    pool_handler()
    
print('running time = '+ f'{time.time()-start_time}')

#Seleciona apenas a coluna x ta tabela de emissão para fazer o gráfico
distx = [point[0] for point in emission]

#Plota e mostra o gráfico
plt.plot(distx, total_photons)
plt.show()